# Install helm

Run the commands below to install helm, for installing postgresql, betydb, and PEcAn. 

In [ ]:
%%bash
mkdir -p ~/.local/bin/ ~/.local/src
curl -fsSL -o ~/.local/bin/get_helm.sh https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3
chmod 700 ~/.local/bin/get_helm.sh
env HELM_INSTALL_DIR=~/.local/bin ~/.local/bin/get_helm.sh --no-sudo
echo DONE

# Install PostgreSQL with helm

Run the commands below to install PostgreSQL relational database with helm. 

In [ ]:
%%bash
helm install postgresql oci://registry-1.docker.io/bitnamicharts/postgresql \
  --set enabled=true \
  --set fullnameOverride=pecan-postgresql \
  --set serviceAccount.enabled=true \
  --set auth.postgresPassword=supersecret
echo DONE

# Install BetyDB with helm

Run the commands below to install BetyDB on top of PostgreSQL for ecological forecasting data. 

In [ ]:
%%bash
git clone https://github.com/computate-org/bety-helm.git ~/.local/src/bety-helm/
helm repo add bitnami https://raw.githubusercontent.com/bitnami/charts/archive-full-index/bitnami
(cd ~/.local/src/bety-helm/ && helm dependency build)
helm install betydb ~/.local/src/bety-helm/  \
  --set postgresql.enabled=false \
  --set postgresql.fullnameOverride=pecan-postgresql \
  --set postgresql.serviceAccount.enabled=true \
  --set postgresql.postgresqlHost=pecan-postgresql \
  --set serviceAccount.enabled=true \
  --set serviceAccount.name=pecan-betydb \
  --set externalSecret=false \
  --set nameOverride=pecan \
  --set fullnameOverride=pecan-betydb
echo DONE

# Install PEcAn with helm

Run the commands below to install PEcAn for ecological forecasting. 

In [ ]:
%%bash
git clone https://github.com/computate-org/pecan-helm.git ~/.local/src/pecan-helm/
helm repo add ncsa https://opensource.ncsa.illinois.edu/charts/
(cd ~/.local/src/pecan-helm/ && helm dependency build)

helm install pecan ~/.local/src/pecan-helm/  \
  -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  --set betydb.enabled=false \
  --set betydb.postgresql.enabled=false \
  --set betydb.postgresql.postgresqlHost=pecan-postgresql \
  --set betydb.postgresql.postgresqlUser=postgres \
  --set betydb.postgresql.postgresqlUsername=postgres \
  --set betydb.postgresql.postgresqlDatabase=postgres \
  --set betydb.postgresql.service.port=5432 \
  --set betydb.service.port=8000 \
  --set clustername=pecan-server \
  --set clusterfqdn=pecan-server-pecan.apps-crc.testing \
  --set initializeData=true \
  --set ingress.enabled=true \
  --set pecan.env.betydb=http://betydb:8000/ \
  --set betydb.betyUser=bety \
  --set betydb.betyDatabase=bety \
  --set betydb.ingress.hosts={betydb.pecan.svc.cluster.local} \
  --set ingress.hosts={pecan-server-pecan.apps-crc.testing}
echo DONE